In [5]:
!pip install nltk

In [8]:
import nltk
import json
import pickle
import random
import numpy as np
from nltk.stem import WordNetLemmatizer
from keras import models, layers, optimizers
from keras.callbacks import EarlyStopping


In [14]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Function to preprocess data

In [10]:

# Function to preprocess data
def preprocess_data():
    words = []
    classes = []
    documents = []
    ignore_words = ['?', '!', '@', '$']

    with open('intents.json', 'r', encoding='utf-8') as data_file:
        intents = json.load(data_file)

    lemmatizer = WordNetLemmatizer()

    for intent in intents['intents']:
        for pattern in intent['patterns']:
            if isinstance(pattern, str):
                w = nltk.word_tokenize(pattern)

                words.extend(w)
                documents.append((w, intent['tag']))

                if intent['tag'] not in classes:
                    classes.append(intent['tag'])
            else:
                print("Skipping non-string pattern:", pattern)
    words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
    words = sorted(list(set(words)))
    classes = sorted(list(set(classes)))


    print(len(words))

    pickle.dump(words, open('words.pkl', 'wb'))
    pickle.dump(classes, open('classes.pkl', 'wb'))

    training_data = []
    output_empty = [0] * len(classes)

    for doc in documents:
        bag = []
        pattern_words = doc[0]
        pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

        for w in words:
            bag.append(1) if w in pattern_words else bag.append(0)

        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1

        training_data.append([bag, output_row])

    random.shuffle(training_data)
    training_data = np.asarray(training_data, dtype="object")
    train_x = list(training_data[:, 0])
    train_y = list(training_data[:, 1])

    return train_x, train_y, classes, words

# Function to create and train model

In [16]:

# Function to create and train model
def create_and_train_model(train_x, train_y, words):
    model = models.Sequential([
        layers.Dense(len(words), input_shape=(len(train_x[0]),), activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(256 , activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(len(train_y[0]), activation='softmax')
    ])

    model.compile(loss="categorical_crossentropy",
                  optimizer=optimizers.Adam(learning_rate=0.001),
                  metrics=["accuracy"])

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    hist = model.fit(np.array(train_x), np.array(train_y),
                     epochs=100,
                     batch_size=24,
                     verbose=1,
                     callbacks=[early_stopping])

    model.save('chatbot_model_5.h5', hist)

# Evaluate model and main function

In [17]:

# Function to evaluate model
def evaluate_model(model, test_x, test_y):
    loss, accuracy = model.evaluate(np.array(test_x), np.array(test_y))
    print("Loss:", loss)
    print("Accuracy:", accuracy)

# Main function
def main():
    train_x, train_y, classes,words = preprocess_data()

    # Splitting data into test and train
    test_size = int(len(train_x) * 0.2)
    test_x = train_x[-test_size:]
    test_y = train_y[-test_size:]
    train_x = train_x[:-test_size]
    train_y = train_y[:-test_size]

    create_and_train_model(train_x, train_y,words)
    model = models.load_model('chatbot_model_5.h5')

    evaluate_model(model, test_x, test_y)


main()

Skipping non-string pattern: {'tag': 'strengthen leg muscles', 'patterns': ['How can I strengthen my leg muscles?', 'Can you recommend leg exercises?', 'What exercises are good for building leg strength?'], 'responses': ['To strengthen your leg muscles, you can include exercises such as squats, lunges, deadlifts, leg presses, and calf raises in your workout routine. These exercises target different muscles in the legs, including the quadriceps, hamstrings, glutes, and calves, and help improve overall leg strength and stability.'], 'context': ['']}
2667
Epoch 1/100
1047/1047 [==============================] - ETA: 0s - loss: 3.1437 - accuracy: 0.4345

1047/1047 [==============================] - 183s 173ms/step - loss: 3.1437 - accuracy: 0.4345
Epoch 2/100
1047/1047 [==============================] - ETA: 0s - loss: 1.7296 - accuracy: 0.6971

1047/1047 [==============================] - 177s 169ms/step - loss: 1.7296 - accuracy: 0.6971
Epoch 3/100
1047/1047 [==============================] - ETA: 0s - loss: 1.4332 - accuracy: 0.7420

1047/1047 [==============================] - 167s 160ms/step - loss: 1.4332 - accuracy: 0.7420
Epoch 4/100
1047/1047 [==============================] - ETA: 0s - loss: 1.2840 - accuracy: 0.7620

1047/1047 [==============================] - 178s 170ms/step - loss: 1.2840 - accuracy: 0.7620
Epoch 5/100
1047/1047 [==============================] - ETA: 0s - loss: 1.1921 - accuracy: 0.7717

...

1047/1047 [==============================] - 156s 149ms/step - loss: 1.0015 - accuracy: 0.8123
Epoch 99/100
1047/1047 [==============================] - ETA: 0s - loss: 1.0228 - accuracy: 0.8088

1047/1047 [==============================] - 158s 151ms/step - loss: 1.0228 - accuracy: 0.8088
Epoch 100/100
1047/1047 [==============================] - ETA: 0s - loss: 0.9849 - accuracy: 0.8151

1047/1047 [==============================] - 157s 150ms/step - loss: 0.9849 - accuracy: 0.8151


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


197/197 [==============================] - 6s 30ms/step - loss: 1.8482 - accuracy: 0.8093
Loss: 1.8482376337051392
Accuracy: 0.8092660307884216
